In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget

import numpy

# Loading and visualizing datasets

## Collection of included datasets

In [ ]:
from pathlib import Path
base_path = Path('sample_data').expanduser()

metadata_paths = {
    'sim_mos2': base_path / Path('simulated_mos2/mos2_0.00_dstep1.0.json'),
    'exp_mos2': base_path / Path('experimental_mos2/mos2_twisted.json'),
    'exp_si': base_path / Path('experimental_si/acq12_20over.json'),
    'sim_si': base_path / Path('simulated_si/Si_110_Sn_300kV_conv25_defocus15_tds_199.70_dstep0.6.json'),
    'exp_prsco3': base_path / Path('experimental_prsco3/PSO.json')
}

for path in metadata_paths.values():
    assert path.exists()

## View dataset metadata

In [ ]:
# load metadata for a dataset
import json

meta_path = metadata_paths['exp_si']

with open(meta_path, 'r') as f:
    metadata = json.load(f)

metadata

## Load raw data

In [ ]:
from phaser.io.empad import load_4d

raw_data = load_4d(meta_path.parent / metadata['raw_filename'])
raw_data.shape

## Visualize raw data

Here, we visualize the sum of the diffraction patterns. This is a position-averaged convergent beam electron diffraction (PACBED) pattern, and can be used to accurately identify crystalline thickness and mistilt

In [ ]:
from phaser.utils.plotting import plot_pacbed

plot_pacbed(raw_data, log=True, diff_step=metadata['diff_step']);

We can also visualize the dataset using virtual images and the raw diffraction patterns. This is useful for identifying data orientation, detector rotation, defocus, and overall dataset quality

In [ ]:
from phaser.utils.plotting import plot_raw

ky = numpy.arange(raw_data.shape[-2], dtype=numpy.float32) - raw_data.shape[-2] / 2.
kx = numpy.arange(raw_data.shape[-1], dtype=numpy.float32) - raw_data.shape[-1] / 2.
ky, kx = numpy.meshgrid(ky, kx, indexing='ij')
k2 = ky**2 + kx**2

# create a virtual detector
# in this case this is a 3 px virtual bright field (vBF) detector
mask = k2 <= 3**2

plot_raw(
    raw_data, mask=mask,
    scan_step=tuple(s*1e10 for s in metadata['scan_step']),
    diff_step=metadata['diff_step'], log=False,
);

# Analyzing sampling metrics

To further assess the quality of the dataset, we can use sampling metrics. These metrics are a useful rule of thumb for identifying optimal acquisition parameters.

In [ ]:
from phaser.utils.optics import calc_metrics
from phaser.utils.plotting import plot_metrics

metrics = calc_metrics(
    voltage=metadata['voltage'], conv_angle=metadata['conv_angle'],
    defocus=metadata['defocus']*1e10, scan_step=metadata['scan_step'][0]*1e10,
    diff_step=metadata['diff_step'], threshold=0.9
)
metrics

In [ ]:
plot_metrics(metrics);

# Running a reconstruction

First, we start a reconstruction manager, which contains reconstruction workers:

In [ ]:
from phaser.web.notebook import Manager
from phaser.plan import ReconsPlan

manager = Manager()
manager.start()

Then, we can submit a job:

Reconstructions are specified in a declarative format, and then processed by the worker to produce a reconstruction

In [ ]:
mos2_meta_path = metadata_paths['sim_mos2']

plan = ReconsPlan.from_data({
    'name': 'mos2_lsqml',

    # how to load the raw data
    'raw_data': {
        'type': 'empad',
        'path': str(mos2_meta_path),
    },

    # add poisson noise to simulated data
    'post_load': [{
        'type': 'poisson',
        'scale': 6.0e+6,
    }],

    'engines': [{
        'type': 'conventional',
        'probe_modes': 2,
        'niter': 200,
        'grouping': 16,
        'noise_model': {
            'type': 'anscombe',
            'eps': 0.1,
        },
        'solver': {
            'type': 'lsqml',
            'gamma': 1.0e-4,
            'illum_reg_object': 1.0e-2,
            'illum_reg_probe': 1.0e-2,
        },
        'group_constraints': [],
        'iter_constraints': [],
        'update_probe': {'after': 1},
        'update_positions': False,
    }]
})

plan

In [ ]:
manager.start_job(plan)

In [ ]:
manager.stop()